In [1]:
import pandas as pd

In [2]:
df = pd.read_json("280_spam_dropped.json")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189762 entries, 0 to 189761
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment       189762 non-null  object
 1   category      189762 non-null  object
 2   channel       189762 non-null  object
 3   video_url     189762 non-null  object
 4   video_id      189762 non-null  object
 5   channel_size  189762 non-null  int64 
 6   identity      189762 non-null  int64 
 7   score         189762 non-null  object
 8   results       189762 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 14.5+ MB


In [3]:
comments = df["comment"].tolist()

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Clean and tokenize comments
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
data = pad_sequences(sequences, maxlen=300)

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(5000, 100, input_length=300))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-08-16 18:19:17.565821: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-08-16 18:19:17.565853: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-08-16 18:19:17.565865: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-08-16 18:19:17.566271: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-16 18:19:17.566680: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
data

array([[   0,    0,    0, ...,    5,    9,   32],
       [   0,    0,    0, ...,    0, 2856,   80],
       [   0,    0,    0, ...,    6,  234,    3],
       ...,
       [   0,    0,    0, ...,  921,    6,   27],
       [   0,    0,    0, ...,  633,  208,    6],
       [   0,    0,    0, ...,    8,  111,  844]], dtype=int32)

In [7]:
sequences

[[314, 608, 5, 9, 32],
 [2856, 80],
 [40, 432, 29, 11, 26, 6, 234, 3],
 [52,
  37,
  19,
  47,
  83,
  11,
  9,
  43,
  4,
  3040,
  3993,
  10,
  1054,
  6,
  438,
  1,
  39,
  366,
  6,
  91,
  187,
  843,
  2695,
  72,
  235,
  1,
  2857,
  1383,
  3,
  1,
  2930,
  30,
  4,
  363,
  336,
  6,
  89,
  9,
  86],
 [18, 3251, 274, 367, 22, 531, 149, 104, 165, 107],
 [34, 1163, 403, 1, 1660, 173, 86],
 [210,
  78,
  18,
  478,
  303,
  280,
  17,
  3746,
  67,
  3,
  79,
  3947,
  12,
  64,
  17,
  1,
  105,
  66,
  1,
  2164,
  72,
  416,
  2254,
  3,
  4408,
  2428,
  6,
  65,
  1,
  128,
  2129,
  11,
  68],
 [2254, 5, 52, 17, 9, 353, 12, 149, 104, 3293, 507],
 [39, 1, 205, 8, 1, 184, 11, 57, 26, 6, 1, 2760, 115],
 [5,
  7,
  1562,
  879,
  41,
  12,
  551,
  10,
  2402,
  23,
  1,
  1144,
  939,
  46,
  49,
  11,
  26,
  6,
  1700,
  1,
  2164,
  43,
  1,
  280],
 [314, 608, 289, 314, 67, 965, 1610],
 [12,
  136,
  4,
  103,
  259,
  2,
  24,
  294,
  12,
  13,
  2092,
  39,
  2304,

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)